In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import squidpy as sq

adata = sq.datasets.visium_hne_adata()
adata

%matplotlib inline

# Task 0

Get familiar with the dataset. Find out which information is contained in `adata`.

The data provided is a 10X Visium measurement of a coronal section of a mouse.<br>
10X Visium is a spot-based sequencing analysis technique.<br>
The dataset is already processed on the count level, so no further pre-processing is required before doing the spatial analyses.

In [2]:
adata

AnnData object with n_obs × n_vars = 2688 × 18078
    obs: 'in_tissue', 'array_row', 'array_col', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'n_counts', 'leiden', 'cluster'
    var: 'gene_ids', 'feature_types', 'genome', 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'cluster_colors', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'spatial', 'umap'
    obsm: 'X_pca', 'X_umap', 'spatial'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

# Task 1

Find genes that show spatial auto-correlation and visually asses their spatial expression distribution.

To do so, you need to first compute the cell-cell neighbor graph.
Based on the technology of the data used (10X Visium), make an informed choice on the neighbor-graph algorithm.

# Task 2

Find cell clusters which are co-localizing, i.e. which have a higher probability to be neighbors.

Plotting the statistics for each cluster might be a bit too laborious.
Instead, you can use some prior knowledge by looking at the cell cluster distribution and derive interesting candidates.

How would you interpret the specific results you see?

# Task 3

What cell clusters are more prevalent in the neighborhood of a specific cell type?
How does this compare to the co-occurence results from task 2?

What biological functions might this imply? What downstream analyses could this trigger?

What does the diagonal mean biologically?